In [1]:
import tensorflow as tf
import numpy as np
import os
import time

# seqeunce.take(5) 문법 변경 문제 해결
tf.enable_eager_execution()

# 세익스피어 파일 다운로드
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [2]:
# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 1115394자


In [3]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 65개


In [5]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [6]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기 / 원하는 만큼 잘라낼 수 있는 함수
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [8]:
# batch 메서드는 이 개별 문자들을 원하는 크기의 시퀀스로 쉽게 변환할 수 있습니다.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
# train / test split
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(2):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
타깃 데이터:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
입력 데이터:  'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
타깃 데이터:  're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [11]:
# 이 벡터의 각 인덱스는 하나의 타임 스텝(time step)으로 처리됩니다. 
# 타임 스텝 0의 입력으로 모델은 "F"의 인덱스를 받고 다음 문자로 "i"의 인덱스를 예측합니다. 
# 다음 타임 스텝에서도 같은 일을 하지만 RNN은 현재 입력 문자 외에 이전 타임 스텝의 컨텍스트(context)를 고려합니다.

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 39 ('a')
  예상 출력: 56 ('r')
   1단계
  입력: 56 ('r')
  예상 출력: 43 ('e')
   2단계
  입력: 43 ('e')
  예상 출력: 1 (' ')
   3단계
  입력: 1 (' ')
  예상 출력: 39 ('a')
   4단계
  입력: 39 ('a')
  예상 출력: 50 ('l')


In [12]:
# 텍스트를 다루기 쉬운 시퀀스로 분리하기 위해 tf.data를 사용했습니다. 
# 그러나 이 데이터를 모델에 넣기 전에 데이터를 섞은 후 배치를 만들어야 합니다.

# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
# (TF 데이터는 무한한 시퀀스와 함께 작동이 가능하도록 설계되었으며,
# 따라서 전체 시퀀스를 메모리에 섞지 않습니다. 대신에,
# 요소를 섞는 버퍼를 유지합니다).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 모델 설계

모델을 정의하려면 tf.keras.Sequential을 사용합니다. 이 간단한 예제에서는 3개의 층을 사용하여 모델을 정의합니다:

- tf.keras.layers.Embedding : 입력층. embedding_dim 차원 벡터에 각 문자의 정수 코드를 매핑하는 훈련 가능한 검색 테이블.
- tf.keras.layers.GRU : 크기가 units = rnn_units인 RNN의 유형(여기서 LSTM층을 사용할 수도 있습니다.
- tf.keras.layers.Dense : 크기가 vocab_size인 출력을 생성하는 출력층.

In [0]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 
                              embedding_dim,
                              batch_input_shape = [batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences = True,
                        stateful = True,
                        recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [38]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2568338510252030418
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14234868150382821748
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7989945687532680196
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14635948153634449088
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [39]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
#   print(example_batch_predictions)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(64, 100, 65) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


In [40]:
print(dataset.take(1))

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [41]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print(sampled_indices)

[44 24 56 47  2 17 47 57 28 42  1  9 27 58 29 43 15 38 52 30  8  7 56 41
 39 22 10 59 49 36 21 27 22 30 19 37 55 54 64  8 15 32 56 18 20 29 39 16
  7 38  8 15 43 35 55  6 58 38 45  0  6  8 13 44 41  1  1 36 30 33 27 26
  2  9 58 58 28 35 27 60 55 53 33 64 51 62 61 26 40 10 42 28 55 59 58 47
 48 38 17 41]


In [42]:
# pretrained model prediction result

print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 't:\nAs mine on hers, so hers is set on mine;\nAnd all combined, save what thou must combine\nBy holy ma'

예측된 다음 문자: 
 'fLri!EisPd 3OtQeCZnR.-rcaJ:ukXIOJRGYqpz.CTrFHQaD-Z.CeWq,tZg\n,.Afc  XRUON!3ttPWOvqoUzmxwNb:dPqutijZEc'


In [43]:
# 손실함수 정의
# 출력값이 로짓의 형태이기 때문에 from_logit = True

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 65)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           4.173877


In [0]:
# tf.keras.Model.compile 메서드를 사용하여 훈련 절차를 설정합니다. 
# 기본 매개변수의 tf.keras.optimizers.Adam과 손실 함수를 사용합니다.

model.compile(optimizer='adam', loss=loss)

In [45]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
#체크포인트 구성
# tf.keras.callbacks.ModelCheckpoint를 사용하여 훈련 중 체크포인트(checkpoint)가 저장되도록 합니다.

# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [47]:
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
172/172 [==============================] - 38s 218ms/step - loss: 2.7615
Epoch 2/10
172/172 [==============================] - 36s 209ms/step - loss: 2.0268
Epoch 3/10
172/172 [==============================] - 36s 208ms/step - loss: 1.7610
Epoch 4/10
172/172 [==============================] - 35s 206ms/step - loss: 1.6101
Epoch 5/10
172/172 [==============================] - 36s 206ms/step - loss: 1.5132
Epoch 6/10
172/172 [==============================] - 35s 206ms/step - loss: 1.4470
Epoch 7/10
172/172 [==============================] - 35s 206ms/step - loss: 1.3970
Epoch 8/10
172/172 [==============================] - 35s 206ms/step - loss: 1.3567
Epoch 9/10
172/172 [==============================] - 35s 206ms/step - loss: 1.3220
Epoch 10/10
172/172 [==============================] - 35s 205ms/step - loss: 1.2886


In [48]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
  num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
  # 온도가 높으면 더 의외의 텍스트가 됩니다.
  # 최적의 세팅을 찾기 위한 실험
  temperature = 1.0

  # 여기에서 배치 크기 == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 배치 차원 제거
      predictions = tf.squeeze(predictions, 0)

      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 예측된 단어를 다음 입력으로 모델에 전달
      # 이전 은닉 상태와 함께
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [51]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: $XV$JJK&Xj&YMPX$xD&M&k&WJXGVJX&JVVj$J&PJ&jXXQDjQ$&P3XQX3xzMXjJjQ3P&XSQXGVK&KQq&$Q&&J&KzQxXjSM&QzXDqV&V$xDxjQ$H&QjxXz$VYMDqxVVzQAGQX&XVzDzYQ3G&DY&z&xXXV&zVjDzjjVzQzDVX&QaXXXQ$&z&VXX&UK$KJjp&$V&DJXQX&JxXVJXLMKYY$SY3&Vj$VzMJXX&&JkzXJQPYJX$KQXjB3$&XXMMj$$JjKV&XQKVDQqjKzKVJ&QxjQMQGj&&xzqQkM3XNVX3jJX&DDJ$&&&PMxjjXQQQXXQJjYQgX3JDjVX&$xVz&&&JZXXXjQjMx&$&Q3VY&JVXj&&XXxqz&DKjZzjGJX&&xx3XV&XxXXqXKKV3JjMY&XzVQVjYKDGMMzXxJzDYQJjzXSYQZqM&MXX$&QqX&NzjX&j3X&QPQXxG$QXXQxkQ&jVJMSYSzJ&Q&XQMjN&X&KjJD3kj$VV3&jCMCpBXXjXMVjYYX&Pz&$Q$QMXX$zJ3X&MM$jz-SGXjKVQjQMVJX?JXBJj$PXJj&Nq&jj$jMNX&X$S&3JXV&V&XPJQ3$XzV3jj&jJ&XQXV&&&KVQQxXjMxXHxMX&KKMxVxMxxzjVXjj&YDX$Q&zH&xjMMXxJzLjXqXXMjM$GO3GXG3QjVUUYV&3&EXD&QXMDXXzz&$jXxNz&Qzj$qXXX3&xDxzKzVzjQ$MMYXXjQX$&&XDz3DJCVz$VX3XQFzjXXkMXJMQX3XjKKYJKX&&zzYQjXXKDVX&3&GzVJ&YXIXTxzVxjYVKxDYDYX$zCzMzXMX$xQ3JJCQ&XKJZJPQXJJMXLXz3Dz3JXGxjF&Kj$MM&&3&jYXQMx$X3zNX&33jX&qjHCQXXjV$z&z$D&M$M&PJjXKQXX$&;XQDVxQ&&QjX&XzJjQX&XXXzQXXzYVVjJX&QQJgJXqXxX&XJqX&HxVKzQXXjVGX&$&&$jVzQxPXXXJXGCSxDzDJ